In [1]:
from __future__ import annotations

import json
import math
import os
from pathlib import Path
from typing import Dict, List

import cv2
import numpy as np
from pycocotools.coco import COCO
from tqdm import tqdm

In [16]:
# -------------------------------------------------
# Configuration – adapt to your project structure
# -------------------------------------------------
SRC_IMG_DIR = Path("../data/rotation/batches/batch_20250703_01/images/default")
ANNOT_PATH = Path("../data/rotation/batches/batch_20250703_01/annotations/instances_train.json")
DEST_PATCH_DIR = Path("../data/rotation/batches/batch_20250703_01/images/boxes")

PAD_RATIO: float = 1.05  # extra context around the box
ANGLES: List[int] = [0, 45, 90, -45, -90, 180]

In [17]:
def ensure_dir(path: Path) -> None:
    """Creates *path* (and parents) if it does not exist."""
    path.mkdir(parents=True, exist_ok=True)


def rectify_patch(img: np.ndarray, cx: float, cy: float, w: float, h: float, theta: float) -> np.ndarray:
    """Return a *sharp* patch rotated back to 0°.

    Parameters
    ----------
    img
        Full RGB/BGR image array.
    cx, cy
        Center of the box (pixels).
    w, h
        Width and height of the box (pixels).
    theta
        Rotation angle in **degrees** (positive = counter‑clockwise).
    """
    # Rotate whole image so the target box becomes axis‑aligned.
    M = cv2.getRotationMatrix2D((cx, cy), -theta, 1.0)
    rotated = cv2.warpAffine(img, M, (img.shape[1], img.shape[0]), flags=cv2.INTER_LINEAR)

    # Crop the patch with a small padding to avoid cutting edges.
    x0 = int(cx - w / 2 * PAD_RATIO)
    y0 = int(cy - h / 2 * PAD_RATIO)
    x1 = int(cx + w / 2 * PAD_RATIO)
    y1 = int(cy + h / 2 * PAD_RATIO)

    return rotated[max(0, y0) : y1, max(0, x0) : x1]

In [ ]:

def rotate_patch(patch: np.ndarray, angle: float) -> np.ndarray:
    """Rotate *patch* by *angle* degrees and return the new image."""
    h, w = patch.shape[:2]
    center = (w / 2, h / 2)

    M = cv2.getRotationMatrix2D(center, angle, 1.0)
    cos, sin = abs(M[0, 0]), abs(M[0, 1])
    new_w = int(h * sin + w * cos)
    new_h = int(h * cos + w * sin)

    # Shift so the image stays centred.
    M[0, 2] += new_w / 2 - center[0]
    
    M[1, 2] += new_h / 2 - center[1]

    return cv2.warpAffine(
        patch, M, (new_w, new_h), flags=cv2.INTER_LINEAR, borderValue=255
    )

In [21]:
# -------------------------------------------------
# COCO helpers
# -------------------------------------------------

def load_coco(annot_path: Path):
    """Return COCO instance and a mapping *image_id ➜ [annotations]*."""
    coco = COCO(str(annot_path))
    img_to_anns: Dict[int, List[dict]] = {}
    for ann in coco.dataset["annotations"]:
        img_to_anns.setdefault(ann["image_id"], []).append(ann)
    return coco, img_to_anns

In [23]:
def process_single_image(
    coco: COCO,
    annotations: Dict[int, List[dict]],
    img_id: int,
    dst_dir: Path = DEST_PATCH_DIR,
):
    """Cut, rectify & replicate patches for one image."""
    info = coco.loadImgs([img_id])[0]
    img = cv2.imread(str(SRC_IMG_DIR / info["file_name"]))
    if img is None:
        raise FileNotFoundError(info["file_name"])

    for i, ann in enumerate(annotations.get(img_id, [])):
        # Handle axis‑aligned (4‑tuple) **and** rotated (5‑tuple) bboxes.
        bbox = ann.get("bbox", [])
        if len(bbox) == 5:
            cx, cy, w, h, theta = bbox
        elif len(bbox) == 4:
            x, y, w, h = bbox
            cx, cy, theta = x + w / 2, y + h / 2, 0
        else:
            continue  # skip malformed ann

        patch = rectify_patch(img, cx, cy, w, h, theta)
        base = f"{Path(info['file_name']).stem}_{i:03d}"

        # Horizontal version
        cv2.imwrite(str(dst_dir / f"{base}_0.png"), patch)

        # Additional angles
        for a in ANGLES:
            if a == 0:
                continue
            cv2.imwrite(
                str(dst_dir / f"{base}_{a:+d}.png"), rotate_patch(patch, a)
            )

In [25]:

def process_batch():
    """Iterate over all images in the dataset."""
    ensure_dir(DEST_PATCH_DIR)
    coco, anns = load_coco(ANNOT_PATH)
    for img_id in tqdm(coco.getImgIds(), desc="Images"):
        process_single_image(coco, anns, img_id)

In [27]:
def _synthetic_test() -> None:
    """Create a synthetic image & run the pipeline without COCO."""
    tmp_dir = Path("_debug")
    ensure_dir(tmp_dir)

    # Synthetic white canvas with one rotated black rectangle
    canvas = np.full((256, 256, 3), 255, np.uint8)
    cv2.rectangle(canvas, (60, 80), (196, 176), (0, 0, 0), 2)
    cx, cy, w, h, theta = 128, 128, 136, 96, 30  # degrees

    rectified = rectify_patch(canvas, cx, cy, w, h, theta)
    cv2.imwrite(str(tmp_dir / "rectified.png"), rectified)

    for a in ANGLES:
        cv2.imwrite(str(tmp_dir / f"rot_{a:+d}.png"), rotate_patch(rectified, a))

    print(f"Test images written to {tmp_dir.resolve()}")

In [28]:

# -------------------------------------------------
# Entry point
# -------------------------------------------------
if __name__ == "__main__":
    # Uncomment ONE of the following lines depending on your intent.

    # -- full dataset run --
    # process_batch()

    # -- quick self‑contained test --
    _synthetic_test()


Test images written to /Users/gerhardkarbeutz/cerpro/ocr-rec-lab/pipeline/_debug
